In [1]:
import pandas as pd
import numpy as np

import sympy
import re

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

# Dados Termodinâmicos

## Importando os Dados Iniciais e Organizando

In [2]:
tabela_qm9 = pd.read_excel('C:\JupyterLab\RedesNeurais&AlgoritmosGeneticos\RedesCarecas_-_Algor-tmicosCalvos\qm9.xlsx')
tabela_qm9.head(5)

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,...,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,...,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,...,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,...,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,...,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028


In [3]:
isomeric_smiles_qm9 = tabela_qm9['smiles'].tolist()

canon_smiles_qm9 = []
for isomeric_smiles in isomeric_smiles_qm9:
    mol = Chem.MolFromSmiles(isomeric_smiles)
    canonical_smiles = Chem.MolToSmiles(mol)
    canon_smiles_qm9.append(canonical_smiles)
    
tabela_qm9.insert(2, "canonical smiles", canon_smiles_qm9)
tabela_qm9.head(5)

,mol_id,smiles,canonical smiles,A,B,C,mu,alpha,homo,lumo,...,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,C,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,...,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,N,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,...,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,O,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,...,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,C#C,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,...,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,C#N,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,...,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028


## Formando a Tabela de Interesse

### Fórmulas Moleculares

In [4]:
## Encontra a entalpia de atomização no QM9 e corrige

formulas_qm9 = []
for i in range(len(canon_smiles_qm9)):
    molecula = Chem.MolFromSmiles(canon_smiles_qm9[i])
    molecula.UpdatePropertyCache()
    formula = Chem.rdMolDescriptors.CalcMolFormula(molecula)
    formulas_qm9.append(formula)

### Número de Átomos

In [5]:
num_atomos_compostos = []  # Lista para armazenar os dicionários de fórmula e quantidade de átomos
padrao = r'([A-Z][a-z]*)(\d*)'

for formula in formulas_qm9:
    quantidade_atomos = {'C': 0, 'H': 0, 'O': 0, 'N': 0, 'P': 0, 'S': 0}  # Dicionário para armazenar a quantidade de cada átomo
    matches = re.findall(padrao, formula)
    for elemento, quantidade in matches:
        if quantidade == '':
            quantidade = 1
        else:
            quantidade = int(quantidade)
        if elemento in quantidade_atomos:
            quantidade_atomos[elemento] += quantidade
    #print(formula, quantidade_atomos)
    num_atomos_compostos.append({'formula': formula, 'quantidade_atomos': quantidade_atomos})

In [6]:
#CHONPS
carbonos = []
hidrogenios = []
oxigenios = []
nitrogenios = []
fosforos = []
enxofres = []

for item in num_atomos_compostos:
    formula = item['formula']
    quantidade_atomos = item['quantidade_atomos']
    # Faça algo com a fórmula e a quantidade de átomos
    a = list(quantidade_atomos.values())
    carbonos.append(a[0])
    hidrogenios.append(a[1])
    oxigenios.append(a[2])
    nitrogenios.append(a[3])
    fosforos.append(a[4])
    enxofres.append(a[5])

### Corrigindo Valores Termodinâmicos

In [7]:
# Correção dos dados do QM9 e obtenção da massa molar com valores obtidos a Temperatura de 298.15 K

## Valores de Entalpia de Atomização (Hartree)
ent_C = -37.844411
ent_H = -0.497912
ent_O = -75.062219
ent_N = -54.581501
ent_P = -99.716370

## Massa Molar (g/mol)
M_C = 12.011
M_H = 1.008
M_O = 15.999
M_N = 14.007
M_P = 30.974

## Valores de Energia Livre de Gibbs

gibbs_C = -37.861317
gibbs_H = -0.510927
gibbs_O = -75.079532
gibbs_N = -54.598897
gibbs_P = -99.733544

## Valores de Energia Interna

interna_C = -37.845355
interna_H = -0.498857
interna_O = -75.063163
interna_N = -54.582445
interna_P = -99.717314

## Valores de Energia Interna

Entalpia_qm9 = []
Gibbs_qm9 = []
Interna_qm9 = []
Massa_molar = []
fator_de_conversaoH_kJ = 2625.5 # De Hartree para kJ/mol
fator_de_conversaoH_kcal = 627.5096

for index in range(len(carbonos)):
    entalpia = tabela_qm9.loc[index, "h298"]
    gibbs = tabela_qm9.loc[index, "g298"]
    interna = tabela_qm9.loc[index, "u298"]
    n_C = carbonos[index]
    n_H = hidrogenios[index]
    n_O = oxigenios[index]
    n_N = nitrogenios[index]
    n_P = fosforos[index]
    
    massa_molar = n_C * M_C + n_H * M_H + n_O * M_O + n_N * M_N + n_P * M_P
    Massa_molar.append(massa_molar)
    
    entalpia_formacao = (-(entalpia - (n_C * ent_C) - (n_H * ent_H) - (n_O * ent_O) - (n_N * ent_N) - (n_P * ent_P)) * fator_de_conversaoH_kcal)
    Entalpia_qm9.append(entalpia_formacao)
    
    energia_livre = (-(gibbs - (n_C * gibbs_C) - (n_H * gibbs_H) - (n_O * gibbs_O) - (n_N * gibbs_N) - (n_P * gibbs_P)) * fator_de_conversaoH_kcal)
    Gibbs_qm9.append(energia_livre)
    
    energia_interna = (-(interna - (n_C * interna_C) - (n_H * interna_H) - (n_O * interna_O) - (n_N * interna_N) - (n_P * interna_P)) * fator_de_conversaoH_kcal)
    Interna_qm9.append(energia_interna)

### Balanceamento de reações

In [8]:
EXPRESSAO_ELEMENTO = re.compile("([A-Z][a-z]?)([0-9]*)")

def parsear_composto(composto):
    """
    Dado um composto químico como Na2SO4,
    retorna um dicionário com a contagem de elementos, como {"Na": 2, "S": 1, "O": 4}
    """
    assert "(" not in composto, "Este analisador não entende subcláusulas"
    return {el: (int(num) if num else 1) for el, num in EXPRESSAO_ELEMENTO.findall(composto)}

def balancear_reacao(compostos_esquerda, compostos_direita):
    compostos_esquerda_parseados = [parsear_composto(composto) for composto in compostos_esquerda]
    compostos_direita_parseados = [parsear_composto(composto) for composto in compostos_direita]

    elementos = sorted(set().union(*compostos_esquerda_parseados, *compostos_direita_parseados))
    indice_elementos = dict(zip(elementos, range(len(elementos))))

    largura = len(compostos_esquerda_parseados) + len(compostos_direita_parseados)
    altura = len(elementos)
    matriz = [[0] * largura for _ in range(altura)]

    for coluna, composto in enumerate(compostos_esquerda_parseados):
        for elemento, quantidade in composto.items():
            linha = indice_elementos[elemento]
            matriz[linha][coluna] = quantidade
    for coluna, composto in enumerate(compostos_direita_parseados, len(compostos_esquerda_parseados)):
        for elemento, quantidade in composto.items():
            linha = indice_elementos[elemento]
            matriz[linha][coluna] = -quantidade

    matriz = sympy.Matrix(matriz)
    coeficientes = matriz.nullspace()[0]
    coeficientes *= sympy.lcm([termo.q for termo in coeficientes])

    balanced_equation = {}
    for i, composto in enumerate(compostos_esquerda):
        balanced_equation[composto] = coeficientes[i]
    for i, composto in enumerate(compostos_direita, len(compostos_esquerda)):
        balanced_equation[composto] = coeficientes[i]

    return balanced_equation

# Exemplo de uso:
compostos_esquerda = ["C2H5O", "O2"]
compostos_direita = ["CO2", "H2O"]

resultado = balancear_reacao(compostos_esquerda, compostos_direita)
print(resultado)

{'C2H5O': 4, 'O2': 11, 'CO2': 8, 'H2O': 10}


### Realizando a Combustão

In [18]:
contador = 0
nao_usados = []
Entalpia_combustão_qm9 = []

ent_o2 = 0 #kcal/mol
ent_co2 = (-187.634176090515 - (ent_C + 2*ent_O)) * fator_de_conversaoH_kcal #kcal/mol adaptado
ent_h2o = 215.159864 #kcal/mol qm9

for index in range(len(formulas_qm9)):
    coef_combustível = 0
    coef_oxigenio = 0
    coef_gas_carbonico = 0
    coef_água = 0
    try:
        entalpia_combustível = Entalpia_qm9[index]
        combustível = formulas_qm9[index]
        reagentes = [combustível, "O2"]
        produtos = ["CO2", "H2O"]
        resultado = balancear_reacao(reagentes, produtos)
        coef_combustível = resultado[f'{combustível}']
        coef_oxigenio = resultado['O2']
        coef_gas_carbonico = resultado['CO2']
        coef_água = resultado['H2O']
        coef_oxigenio = coef_oxigenio/coef_combustível
        coef_gas_carbonico = coef_gas_carbonico/coef_combustível
        coef_água = coef_água/coef_combustível
        coef_combustível = coef_combustível/coef_combustível
        entalpia_de_combustão = (ent_h2o * coef_água + ent_co2 * coef_gas_carbonico) - (entalpia_combustível * coef_combustível + ent_o2 * coef_oxigenio)
        Entalpia_combustão_qm9.append(round(float(entalpia_de_combustão), 6))
        
    except:
        contador += 1
        nao_usados.append(combustível)
        Entalpia_combustão_qm9.append(0)
    
print(f'Não foi possível computar {contador} compostos')
print(f'Foi possível computar {len(formulas_qm9) - contador} compostos')

Não foi possível computar 83207 compostos
Foi possível computar 50678 compostos


### Montando a Tabela

In [19]:
# Criação do Dataframe

dic_entalpias = {
    "Fórmula" : formulas_qm9,
    "N° de Carbonos": carbonos,
    "N° de Hidrogênios": hidrogenios,
    "N° de Oxigênios": oxigenios,
    "N° de Nitrogênios": nitrogenios,
    "N° de Fósforos": fosforos,
    "Isomeric Smiles": isomeric_smiles_qm9,
    "Smiles": canon_smiles_qm9,
    "Massa Molar (g/mol)": Massa_molar,
    "Entalpia de Combustão (kcal/mol)": Entalpia_combustão_qm9,
    "Entalpia de Formação (kcal/mol)": Entalpia_qm9,
    "Energia Interna (kcal)": Interna_qm9,
    "Energia Livre de Gibbs (kcal)": Gibbs_qm9,
}
tabela_entalpias = pd.DataFrame(dic_entalpias)
tabela_entalpias.head(50)

,Fórmula,N° de Carbonos,N° de Hidrogênios,N° de Oxigênios,N° de Nitrogênios,N° de Fósforos,Isomeric Smiles,Smiles,Massa Molar (g/mol),Entalpia de Combustão (kcal/mol),Entalpia de Formação (kcal/mol),Energia Interna (kcal),Energia Livre de Gibbs (kcal)
0,CH4,1,4,0,0,0,C,C,16.043,239.315162,401.015030,398.643671,372.472128
1,H3N,0,3,0,1,0,N,N,17.031,0.000000,280.399527,278.620537,259.339050
2,H2O,0,2,1,0,0,O,O,18.015,-0.000000,215.159864,213.974499,201.407364
3,C2H2,2,2,0,0,0,C#C,C#C,26.038,246.164372,389.016419,387.238057,365.801074
4,CHN,1,1,0,1,0,C#N,C#N,27.026,0.000000,304.091780,302.907042,288.720305
5,CH2O,1,2,1,0,0,C=O,C=O,30.026,62.878915,362.291413,360.513050,340.464746
6,C2H6,2,6,0,0,0,CC,CC,30.070,385.639048,679.861471,675.711122,626.927899
7,CH4O,1,4,1,0,0,CO,CO,32.042,153.010001,487.320190,484.355835,450.124559
8,C3H4,3,4,0,0,0,CC#C,C#CC,40.065,382.813316,677.537803,673.981078,631.347449
9,C2H3N,2,3,0,1,0,CC#N,CC#N,41.053,0.000000,595.858016,592.894288,557.126241


In [20]:
tabela_entalpias.to_excel("Tabela_QM9_Termodinamica.xlsx", index=False)